Deep Learning
=============

Assignment 6
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [2]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
    """Download a file if not present, and make sure it's the right size."""
    if not os.path.exists(filename):
        filename, _ = urlretrieve(url + filename, filename)
    statinfo = os.stat(filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified %s' % filename)
    else:
        print(statinfo.st_size)
        raise Exception('Failed to verify ' + filename + '. Can you get to it with a browser?')
    return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [3]:
def read_data(filename):
    with zipfile.ZipFile(filename) as f:
        name = f.namelist()[0]
        data = tf.compat.as_str(f.read(name))
    return data
  
text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


Create a small validation set.

In [4]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


Utility functions to map characters to vocabulary IDs and back.

In [5]:
VOCAB = len(string.ascii_lowercase) + 1 # [a-z] + ' '
firstLetter = ord(string.ascii_lowercase[0])

def char2id(char):
    if char in string.ascii_lowercase:
        return ord(char) - firstLetter + 1
    elif char == ' ':
        return 0
    else:
        print('Unexpected character: %s' % char)
    return 0

def id2char(dictid):
    if dictid > 0:
        return chr(dictid + firstLetter - 1)
    else:
        return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Unexpected character: ï
1 26 0 0
a z  


Function to generate a training batch for the LSTM model.

In [6]:
BATCH = 64
UNROLLINGS =10

class BatchGenerator(object):
    def __init__(self, text, BATCH, UNROLLINGS):
        self._text = text
        self._text_size = len(text)
        self._batch_size = BATCH
        self._num_unrollings = UNROLLINGS
        segment = self._text_size // BATCH
        self._cursor = [ offset * segment for offset in range(BATCH)]
        self._last_batch = self._next_batch()
  
    def _next_batch(self):
        """Generate a single batch from the current cursor position in the data."""
        batch = np.zeros(shape=(self._batch_size, VOCAB), dtype=np.float)
        for b in range(self._batch_size):
            batch[b, char2id(self._text[self._cursor[b]])] = 1.0
            self._cursor[b] = (self._cursor[b] + 1) % self._text_size
        return batch
  
    def next(self):
        """Generate the next array of batches from the data. The array consists of
        the last batch of the previous array, followed by num_unrollings new ones.
        """
        batches = [self._last_batch]
        for step in range(self._num_unrollings):
            batches.append(self._next_batch())
        self._last_batch = batches[-1]
        return batches

def characters(probabilities):
    """Turn a 1-hot encoding or a probability distribution over the possible
    characters back into its (most likely) character representation."""
    return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
    """Convert a sequence of batches back into their (most likely) string
    representation."""
    s = [''] * batches[0].shape[0]
    for b in batches:
        s = [''.join(x) for x in zip(s, characters(b))]
    return s

train_batches = BatchGenerator(train_text, BATCH, UNROLLINGS)
valid_batches = BatchGenerator(valid_text, 1, 1)

print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))

['ons anarchi', 'when milita', 'lleria arch', ' abbeys and', 'married urr', 'hel and ric', 'y and litur', 'ay opened f', 'tion from t', 'migration t', 'new york ot', 'he boeing s', 'e listed wi', 'eber has pr', 'o be made t', 'yer who rec', 'ore signifi', 'a fierce cr', ' two six ei', 'aristotle s', 'ity can be ', ' and intrac', 'tion of the', 'dy to pass ', 'f certain d', 'at it will ', 'e convince ', 'ent told hi', 'ampaign and', 'rver side s', 'ious texts ', 'o capitaliz', 'a duplicate', 'gh ann es d', 'ine january', 'ross zero t', 'cal theorie', 'ast instanc', ' dimensiona', 'most holy m', 't s support', 'u is still ', 'e oscillati', 'o eight sub', 'of italy la', 's the tower', 'klahoma pre', 'erprise lin', 'ws becomes ', 'et in a naz', 'the fabian ', 'etchy to re', ' sharman ne', 'ised empero', 'ting in pol', 'd neo latin', 'th risky ri', 'encyclopedi', 'fense the a', 'duating fro', 'treet grid ', 'ations more', 'appeal of d', 'si have mad']
['ists advoca', 'ary governm', 'hes nat

In [7]:
def logprob(predictions, labels):
    """Log-probability of the true labels in a predicted batch."""
    predictions[predictions < 1e-10] = 1e-10
    return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
    """Sample one element from a distribution assumed to be an array of normalized
    probabilities.
    """
    r = random.uniform(0, 1)
    s = 0
    for i in range(len(distribution)):
        s += distribution[i]
        if s >= r:
            return i
    return len(distribution) - 1

def sample(prediction):
    """Turn a (column) prediction into 1-hot encoded samples."""
    p = np.zeros(shape=[1, VOCAB], dtype=np.float)
    p[0, sample_distribution(prediction[0])] = 1.0
    return p

def random_distribution():
    """Generate a random column of probabilities."""
    b = np.random.uniform(0.0, 1.0, size=[1, VOCAB])
    return b / np.sum(b, 1)[:,None]

## LSTM

In [8]:
NODES = 64
BATCH = 64
UNROLLINGS = 10

### Input data

In [9]:
data = [tf.placeholder(tf.float32, shape=[BATCH, VOCAB]) for _ in range(UNROLLINGS + 1)]
trainX = data[:UNROLLINGS]
trainY = data[1:]  # labels are inputs shifted by one time step.

<img src="../Collection/img/lstm.png" style="width: 600px;float:left"/>

### Forget Gate

<img src="../Collection/img/forget.png" style="width: 600px;float:left"/>

In [10]:
wfH = tf.Variable(tf.truncated_normal([NODES, NODES], -0.1, 0.1))
wfX = tf.Variable(tf.truncated_normal([VOCAB, NODES], -0.1, 0.1))
bf = tf.Variable(tf.zeros([1, NODES]))

### Input Gate and Memory Cell

<img src="../Collection/img/input.png" style="width: 600px;float:left"/>

In [11]:
wiH = tf.Variable(tf.truncated_normal([NODES, NODES], -0.1, 0.1))
wiX = tf.Variable(tf.truncated_normal([VOCAB, NODES], -0.1, 0.1))
bi = tf.Variable(tf.zeros([1, NODES]))

In [12]:
wcH = tf.Variable(tf.truncated_normal([NODES, NODES], -0.1, 0.1))
wcX = tf.Variable(tf.truncated_normal([VOCAB, NODES], -0.1, 0.1))
bc = tf.Variable(tf.zeros([1, NODES]))

### Output Gate

<img src="../Collection/img/output.png" style="width: 600px;float:left"/>

In [13]:
woH = tf.Variable(tf.truncated_normal([NODES, NODES], -0.1, 0.1))
woX = tf.Variable(tf.truncated_normal([VOCAB, NODES], -0.1, 0.1))
bo = tf.Variable(tf.zeros([1, NODES]))

<img src="../Collection/img/update.png" style="width: 600px;float:left"/>

In [14]:
def LSTM(inputX, outputH, state):
    fGate = tf.sigmoid(tf.matmul(outputH, wfH) + tf.matmul(inputX, wfX) + bf)
    iGate = tf.sigmoid(tf.matmul(outputH, wiH) + tf.matmul(inputX, wiX) + bi)
    oGate = tf.sigmoid(tf.matmul(outputH, woH) + tf.matmul(inputX, woX) + bo)
    update = tf.tanh(tf.matmul(outputH, wcH) + tf.matmul(inputX, wcX) + bc)
    state = fGate * state + iGate * tf.tanh(update)
    output = oGate * tf.tanh(state)
    return output, state

In [15]:
savedOutput = tf.Variable(tf.zeros([BATCH, NODES]), trainable=False)
savedState = tf.Variable(tf.zeros([BATCH, NODES]), trainable=False)

# Unrolled LSTM loop.
outputs = list()
outputH = savedOutput
state = savedState
for inputX in trainX:
    output, state = LSTM(inputX, outputH, state)
    outputs.append(output)

In [16]:
# Classifier weights and biases.
w = tf.Variable(tf.truncated_normal([NODES, VOCAB], -0.1, 0.1))
b = tf.Variable(tf.zeros([VOCAB]))

# State saving across unrollings.
with tf.control_dependencies([savedOutput.assign(outputH), savedState.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf.concat(trainY, 0), logits=logits))

In [17]:
# Optimizer.
global_step = tf.Variable(0)
learning_rate = tf.train.exponential_decay(10.0, global_step, 5000, 0.1, staircase=True)
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
gradients, v = zip(*optimizer.compute_gradients(loss))
gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=global_step)

# Predictions.
train_prediction = tf.nn.softmax(logits)

# Sampling and validation eval: batch 1, no unrolling.
sample_input = tf.placeholder(tf.float32, shape=[1, VOCAB])
saved_sample_output = tf.Variable(tf.zeros([1, NODES]))
saved_sample_state = tf.Variable(tf.zeros([1, NODES]))
reset_sample_state = tf.group(saved_sample_output.assign(tf.zeros([1, NODES])), saved_sample_state.assign(tf.zeros([1, NODES])))
sample_output, sample_state = LSTM(sample_input, saved_sample_output, saved_sample_state)
with tf.control_dependencies([saved_sample_output.assign(sample_output), saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

## Training

In [18]:
STEPS = 7001
summary_frequency = 100

init = tf.global_variables_initializer()
session = tf.Session()
session.run(init)

print('Initialized')
mean_loss = 0
for step in range(STEPS):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(UNROLLINGS + 1):
        feed_dict[data[i]] = batches[i]
    _, l, predictions, lr = session.run([optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
        if step > 0:
            mean_loss = mean_loss / summary_frequency
            # The mean loss is an estimate of the loss over the last few batches.
        print('Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
        mean_loss = 0
        labels = np.concatenate(list(batches)[1:])
        print('Minibatch perplexity: %.2f' % float(np.exp(logprob(predictions, labels))))
        if step % (summary_frequency * 10) == 0:
            # Generate some samples.
            print('=' * 80)
            for _ in range(5):
                feed = sample(random_distribution())
                sentence = characters(feed)[0]
                session.run(reset_sample_state)
                for _ in range(79):
                    prediction = session.run(sample_prediction, {sample_input: feed})
                    feed = sample(prediction)
                    sentence += characters(feed)[0]
                print(sentence)
            print('=' * 80)
        # Measure validation set perplexity.
        session.run(reset_sample_state)
        valid_logprob = 0
        for _ in range(valid_size):
            b = valid_batches.next()
            predictions = session.run(sample_prediction, {sample_input: b[0]})
            valid_logprob = valid_logprob + logprob(predictions, b[1])
        print('Validation set perplexity: %.2f' % float(np.exp(valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.292724 learning rate: 10.000000
Minibatch perplexity: 26.92
m meym ipyuage dznds ozwe bs ockfcxpznb wuznmitrrlfahraje  pyjbjrdfj  yfggint ff
ebaqbie woqtaygw  druf assovzoxdehz dkcviol  gdxc fhrqrrtdx tmieyxuiii fidyhrevm
gijuohhdftdi eltwdbkcqaqn  smmy  aenf xnascuabxvrneyd xitpy utcbnz hlaayvsiirbse
zgdaiwqtrfilos o aavy  tiekdudeuthr isxxjesxsfe ni  rywhud fzjictyn jh   y trlth
wawwcdabneoj cidgsnfo ug lce cl ick kpdvnpbatqpdd tase jfp rlbfmorqs  jvunm ve i
Validation set perplexity: 20.15
Average loss at step 100: 2.661539 learning rate: 10.000000
Minibatch perplexity: 12.69
Validation set perplexity: 13.28
Average loss at step 200: 2.408464 learning rate: 10.000000
Minibatch perplexity: 10.62
Validation set perplexity: 11.21
Average loss at step 300: 2.353877 learning rate: 10.000000
Minibatch perplexity: 9.69
Validation set perplexity: 10.98
Average loss at step 400: 2.309455 learning rate: 10.000000
Minibatch perplexity: 9.69
Validation set 

---
Problem 1
---------

You might have noticed that the definition of the LSTM cell involves 4 matrix multiplications with the input, and 4 matrix multiplications with the output. Simplify the expression by using a single matrix multiply for each, and variables that are 4 times larger.

---

---
Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---

---
Problem 3
---------

(difficult!)

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is:

    the quick brown fox
    
the model should attempt to output:

    eht kciuq nworb xof
    
Refer to the lecture on how to put together a sequence-to-sequence model, as well as [this article](http://arxiv.org/abs/1409.3215) for best practices.

---